In [1]:
%config InlineBackend.figure_formats = ['svg']
import sys
sys.path.insert(1, '../')

import matplotlib.pyplot as plt
from os import path

from deployment_instance.Result import ExperimentResult
from src.ingest_data import ingest_experiment_results
from src.query_data import get_exfiltration_time_df, get_data_exfiltration_cdf, total_control_host_capture_times
from statistics import mean
import seaborn as sns
# set seaborn font size
sns.set_theme(font_scale=1.5)
sns.set_style("whitegrid", {'axes.grid' : False})

import pandas as pd

In [2]:
# Equifax d10_h0
equifax_standalone_d10_h0 = ingest_experiment_results(path.join('data/EquifaxLarge/standalone/equifax/equifax_standalone_d10_h0'))
dfs_standalone_d10_h0 = ingest_experiment_results(path.join('data/EquifaxLarge/standalone/dfs/dfs_standalone_d10_h0'))
targeted_standalone_d10_h0 = ingest_experiment_results(path.join('data/EquifaxLarge/standalone/targeted/targeted_standalone_d10_h0'))
persistentdfs_standalone_d10_h0 = ingest_experiment_results(path.join('data/EquifaxLarge/standalone/persistent_dfs/persistentdfs_standalone_d10_h0'))
darkside_standalone_d10_h0 = ingest_experiment_results(path.join('data/EquifaxLarge/standalone/darkside/darkside_standalone_d10_h0'))

equifax_standalone_d10_data = {}
equifax_standalone_d10_data.update(equifax_standalone_d10_h0)
equifax_standalone_d10_data.update(dfs_standalone_d10_h0)
equifax_standalone_d10_data.update(targeted_standalone_d10_h0)
equifax_standalone_d10_data.update(persistentdfs_standalone_d10_h0) 
equifax_standalone_d10_data.update(darkside_standalone_d10_h0)

# Equifax d10_h50
equifax_standalone_d10_h50 = ingest_experiment_results(path.join('data/EquifaxLarge/standalone/equifax/equifax_standalone_d10_h50'))
dfs_standalone_d10_h50 = ingest_experiment_results(path.join('data/EquifaxLarge/standalone/dfs/dfs_standalone_d10_h50'))
targeted_standalone_d10_h50 = ingest_experiment_results(path.join('data/EquifaxLarge/standalone/targeted/targeted_standalone_d10_h50'))
persistentdfs_standalone_d10_h50 = ingest_experiment_results(path.join('data/EquifaxLarge/standalone/persistent_dfs/persistentdfs_standalone_d10_h50'))
darkside_standalone_d10_h50 = ingest_experiment_results(path.join('data/EquifaxLarge/standalone/darkside/darkside_standalone_d10_h50'))

equifax_standalone_d10_h50_data = {}
equifax_standalone_d10_h50_data.update(equifax_standalone_d10_h50)
equifax_standalone_d10_h50_data.update(dfs_standalone_d10_h50)
equifax_standalone_d10_h50_data.update(targeted_standalone_d10_h50)
equifax_standalone_d10_h50_data.update(persistentdfs_standalone_d10_h50)
equifax_standalone_d10_h50_data.update(darkside_standalone_d10_h50)


In [3]:
equifax_standalone_d10_data_times = get_exfiltration_time_df(equifax_standalone_d10_data, 48)
equifax_standalone_d10_data_times.groupby('experiment').agg({'time_per_file': ['mean', 'std'], 'percent_files_exfiltrated': ['mean', 'std']})

time_per_file            \
                                         mean       std   
experiment                                                
baseline_standalone_d10_h0           0.883508  0.041200   
darkside_standalone_d10_h0           0.884415  0.011239   
dfs_standalone_d10_h0                4.802488  0.935535   
persistentdfs_standalone_d10_h0      7.738779  2.484619   
targeted_standalone_d10_h0           0.000000  0.000000   

                                percent_files_exfiltrated            
                                                     mean       std  
experiment                                                           
baseline_standalone_d10_h0                     100.000000  0.000000  
darkside_standalone_d10_h0                     100.000000  0.000000  
dfs_standalone_d10_h0                           31.250000  4.885850  
persistentdfs_standalone_d10_h0                 20.833333  8.715209  
targeted_standalone_d10_h0                       0.000000  0.000000

In [4]:
equifax_standalone_d10_h50_data_times = get_exfiltration_time_df(equifax_standalone_d10_h50_data, 48)
equifax_standalone_d10_h50_data_times.groupby('experiment').agg({'time_per_file': ['mean', 'std'], 'percent_files_exfiltrated': ['mean', 'std']})

time_per_file            \
                                          mean       std   
experiment                                                 
baseline_standalone_d10_h50           0.918099  0.059972   
darkside_standalone_d10_h50           0.251892  0.466766   
dfs_standalone_d10_h50                0.000000  0.000000   
persistentdfs_standalone_d10_h50      0.000000  0.000000   
targeted_standalone_d10_h50           0.000000  0.000000   

                                 percent_files_exfiltrated             
                                                      mean        std  
experiment                                                             
baseline_standalone_d10_h50                          100.0   0.000000  
darkside_standalone_d10_h50                           25.0  46.291005  
dfs_standalone_d10_h50                                 0.0   0.000000  
persistentdfs_standalone_d10_h50                       0.0   0.000000  
targeted_standalone_d10_h50                            0.0   0.000000

In [11]:
# ICS d10_h0
equifax_standalone_d10_h0 = ingest_experiment_results(path.join('data/ICS/standalone/equifax/equifax_standalone_d10_h0'))
dfs_standalone_d10_h0 = ingest_experiment_results(path.join('data/ICS/standalone/dfs/dfs_standalone_d10_h0'))
targeted_standalone_d10_h0 = ingest_experiment_results(path.join('data/ICS/standalone/targeted/targeted_standalone_d10_h0'))
persistentdfs_standalone_d10_h0 = ingest_experiment_results(path.join('data/ICS/standalone/persistent_dfs/persistentdfs_standalone_d10_h0'))
darkside_standalone_d10_h0 = ingest_experiment_results(path.join('data/ICS/standalone/darkside/darkside_standalone_d10_h0'))

ics_standalone_d10_h50_data = {}
ics_standalone_d10_h50_data.update(equifax_standalone_d10_h50)
ics_standalone_d10_h50_data.update(dfs_standalone_d10_h50)
ics_standalone_d10_h50_data.update(targeted_standalone_d10_h50)
ics_standalone_d10_h50_data.update(persistentdfs_standalone_d10_h50)
ics_standalone_d10_h50_data.update(darkside_standalone_d10_h50)

In [12]:
ics_standalone_d10_data_times = total_control_host_capture_times(ics_standalone_d10_h50_data, 'ics')
ics_standalone_d10_data_times.groupby('experiment').agg({'percent_hosts_infected': ['mean', 'std']})

AttributeError: 'str' object has no attribute 'name'

In [13]:
darkside_standalone_d10_h0

{'2024-11-03_05-15-12': ExperimentResult(scenario=Scenario(name='darkside_standalone_d10_h0', defender=DefenderInformation(telemetry='NoTelemetry', strategy='StaticStandalone', capabilities={'DeployDecoy': 10, 'HoneyService': 0, 'RestoreServer': 0, 'HoneyCredentials': 0}), attacker=AttackerInformation(name='darkside'), deployment_instance=DeploymentInstanceInformation(name='ICSEnvironment')), experiment_time=1343.8567917346954, execution_time=904.6152412891388, setup_time=439.24155044555664, flags_captured=[], data_exfiltrated=[], hosts_infected=[HostInfected(name='decoy-2', time_infected=50.383084774017334), HostInfected(name='manage-a-0', time_infected=87.54387950897217), HostInfected(name='decoy-4', time_infected=102.59092688560486), HostInfected(name='decoy-7', time_infected=122.66044807434082), HostInfected(name='sensor-17', time_infected=189.0520040988922), HostInfected(name='sensor-13', time_infected=199.13599514961243), HostInfected(name='sensor-14', time_infected=214.205048084

In [14]:
ics_standalone_d10_h50_data

{'ad983bfc-4250-4603-8546-a7d606b8a6e6': ExperimentResult(scenario=Scenario(name='baseline_standalone_d10_h50', defender=DefenderInformation(telemetry='NoTelemetry', strategy='StaticStandalone', capabilities={'DeployDecoy': 10, 'HoneyService': 0, 'RestoreServer': 0, 'HoneyCredentials': 50}), attacker=AttackerInformation(name='EquifaxBaselineAttacker'), deployment_instance=DeploymentInstanceInformation(name='EquifaxLarge')), experiment_time=4003.1972744464874, execution_time=3046.8084046840668, setup_time=956.3888697624207, flags_captured=[], data_exfiltrated=[DataExfiltrated(name='data_database_36.json', time_exfiltrated=411.11012959480286), DataExfiltrated(name='data_database_9.json', time_exfiltrated=454.11012959480286), DataExfiltrated(name='data_database_13.json', time_exfiltrated=496.11012959480286), DataExfiltrated(name='data_database_3.json', time_exfiltrated=538.1101295948029), DataExfiltrated(name='data_database_46.json', time_exfiltrated=601.1101295948029), DataExfiltrated(na